In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

import glob

In [2]:
# Machine-learning specific imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
## Import symbol list 
path = Path('../Resources/06_01_ML_symbol_success_list.pkl')
symbol_list = load_obj(path)
len(symbol_list)

770

In [5]:
# ## Test import code for the dataframe 

# symbol = 'GME'
# path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')

# data = load_obj(path)
# import_df = data[symbol]
# df = import_df.copy()
# df

In [6]:
## Takes one symbol and runs model data. 
## Call function for each individal symbol. No return data.  

def mean_squared_model(
    symbol,
    model_count = 5, ## Number of times model runs before saving the best one. Should be multiple of 3
    validation_split_value=0.3,
    epochs_value=400,
    units1 = 8,
    units2 = 8,
    ):
    
    path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
      
    data = load_obj(path)
    import_df = data[symbol]
    df = import_df.copy()
    #data_dict[key] = df
        
    X = df.drop(columns={'close','adjClose'}).values
    y = df['close'].values
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)    
    
    num_of_inputs = 20
    num_of_outputs= 1
    layers = 1 ## Run the first model with one layer 

    for i in range(model_count):
        ## Create Neural Network 

        # Define the model - deep neural network with two layers
        nn = Sequential()

        # First hidden layer
        nn.add(Dense(units=units1, input_dim=num_of_inputs, activation="relu"))

        # Second hidden layer
        nn.add(Dense(units=units2, activation="relu"))

        # Output layer
        nn.add(Dense(units=num_of_outputs, activation="linear"))

        # Compile the model
        nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model = nn.fit(X, y, 
                          validation_split=validation_split_value, 
                          epochs=epochs_value, 
                          verbose=0)
            
        ## End of if/elif

        model_loss, model_accuracy = nn.evaluate(X, y, verbose=0)
        
        ## Save first model 
        if i == 0:
            model_accuracy_high= model_accuracy
            model_accuracy_low = model_accuracy
            symbol_accuracy_dict_high[symbol] = model_accuracy_high
            symbol_accuracy_dict_low[symbol] = model_accuracy_low
            # Save model as JSON
            nn_json = nn.to_json()

            file_path = Path('../Model_Data_high_loss/'+symbol+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = ('../Model_Data_high_loss/'+symbol+'_model_weights.h5')
            nn.save_weights(file_path)
            
            file_path = Path('../Model_Data_low_loss/'+symbol+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = ('../Model_Data_low_loss/'+symbol+'_model_weights.h5')
            nn.save_weights(file_path)
        ## Rewrite saved model if accuracy is higher or lower 
        else:
            if model_accuracy > model_accuracy_high:
                ## Rewrite values 
                model_accuracy_high = model_accuracy
                symbol_accuracy_dict_high[symbol] = model_accuracy_high
                
                ## Rewrite saved files 
                nn_json = nn.to_json()

                file_path = Path('../Model_Data_high_loss/'+symbol+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = ('../Model_Data_high_loss/'+symbol+'_model_weights.h5')
                nn.save_weights(file_path)
            elif model_accuracy < model_accuracy_low:
                ## Rewrite values 
                model_accuracy_low = model_accuracy
                symbol_accuracy_dict_low[symbol] = model_accuracy_low
                
                # Rewrite saved files 
                nn_json = nn.to_json()

                file_path = Path('../Model_Data_low_loss/'+symbol+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = ('../Model_Data_low_loss/'+symbol+'_model_weights.h5')
                nn.save_weights(file_path)
    ## End of for loop 
    ## Return nothing 

In [7]:
## Create empty dicts for accuracy data 
symbol_accuracy_dict_low = {}
symbol_accuracy_dict_high = {}

## Run for loop through entire key list
## Create model for all symbols in list 
for symbol in symbol_list:
    try:
        mean_squared_model(symbol)
    except:
        continue
    
## Export accuracy_dicts after for loop 
path = ('../Resources/symbol_accuracy_dict_low.pkl')
save_obj(symbol_accuracy_dict_low,path)
path = ('../Resources/symbol_accuracy_dict_high.pkl')
save_obj(symbol_accuracy_dict_high,path)